In [1]:
# Get the training data
# Clean/preprocess/transform the data
# Train a machine learning model
# Evaluate and optimise the model
# Clean/preprocess/transform new data
# Fit the model on new data to make predictions.

In [2]:
import pandas as pd
import numpy as np
import load_data
from sklearn.model_selection import train_test_split
import imblearn
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import accuracy_score,classification_report,recall_score,confusion_matrix, roc_auc_score, precision_score, f1_score, roc_curve, auc, plot_confusion_matrix,plot_roc_curve
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestRegressor

from catboost import CatBoostClassifier

pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

In [3]:
#Load Data
mypath = "../data/"
mydata = load_data.get_file_names(mypath)
data_files = load_data.load_copy_data(mydata, mypath)

data = data_files['data_merged']

file name: X_train_des
file name: y_test_des
file name: X_test_des
file name: train_month_1
file name: data_merged
file name: train_month_2
file name: test_month_1
file name: test_month_3
file name: test_month_2
file name: y_train_des
file name: train_month_3_with_target


In [4]:
#Construct test, train set
X = data.drop('target',axis=1)
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [5]:
 # summarize class distribution
print(Counter(y_train))
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy=0.5)
# define undersample strategy
undersample = RandomUnderSampler(sampling_strategy=0.5)
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X_train, y_train)
X_under, y_under = undersample.fit_resample(X_train, y_train)
# summarize class distribution
print(Counter(y_over))
print(Counter(y_under))

Counter({0: 49408, 1: 1549})
Counter({0: 49408, 1: 24704})
Counter({0: 3098, 1: 1549})


In [6]:
# Pre-process individual columns

# Change dates to inbetween years
class InbetweenDays(BaseEstimator):
    
    def __init__(self):
        pass
    
    def fit(self, documents, y=None):
        return self
    
    def transform(self, x_dataset):
        list_dates = ['customer_since_all_x', 'customer_since_bank_x', 'customer_birth_date_x']

        #Convert date columns into datetime format
        x_dataset['base_dt'] = pd.to_datetime('2018-01-01')
        x_dataset[list_dates] = x_dataset[list_dates].apply(pd.to_datetime)

        for col in list_dates:
            x_dataset[col] = abs(x_dataset['base_dt'].dt.year - x_dataset[col].dt.year)

        #Drop columns (base_dt)
        x_dataset = x_dataset.drop('base_dt', axis=1)
    
        return x_dataset
    
# Change Children Status to binary
class ChildrenStatus(BaseEstimator):
    
    def __init__(self):
        pass
    
    def fit(self, documents, y=None):
        return self
    
    def transform(self, x_dataset):
        for column in ['customer_children', 'customer_children_y', 'customer_children_x']:
            x_dataset[column].replace(['mature', 'young', 'onebaby', 'adolescent', 'preschool', 'grownup'], 'yes', inplace=True)
            x_dataset[column].fillna('no', inplace=True)
    
        return x_dataset

# Categorize Area Code by 1000s
class AreaCode(BaseEstimator):
    
    def __init__(self):
        pass
    
    def fit(self, documents, y=None):
        return self
    
    def transform(self, x_dataset):
        # Bin area codes by 1000s 
        labels = ["{}_area_code".format(i) for i in range(0, 10000, 1000)]
        x_dataset['area_cat'] = pd.cut(x_dataset['customer_postal_code_x'], range(0, 10005, 1000), right=False, labels=labels)
        
        return x_dataset

# Get % Change from time point 1 to 2, 2 to 3, 1 to 3
class PercetChange(BaseEstimator):

    # bal_insurance_21: balance on "tak 21" life insurance
    # bal_insurance_23: balance on "tak 23" life insurance
    # bal_personal_loan: outstanding balance on personal loans
    # bal_mortgage_loan: outstanding balance on mortgage loans
    # bal_current_account: balance on current (checkings) accounts
    # bal_pension_saving: balance on pension (retirement) savings accounts
    # bal_savings_account: balance on savings accounts
    # bal_current_account_starter: balance on starter current (checkings) accounts
    # bal_savings_account_starter: balance on starter savings accounts
    
    def __init__(self):
        pass
    
    def fit(self, documents, y=None):
        return self
    
    def transform(self, x_dataset):

        def percentage_change(col1,col2):
            return ((col2 - col1) / col1) * 100

        list_balances = [['bal_insurance_21_x', 'bal_insurance_21_y', 'bal_insurance_21'],
                         ['bal_insurance_23_x', 'bal_insurance_23_y', 'bal_insurance_23'],
                         ['bal_personal_loan_x', 'bal_personal_loan_y', 'bal_personal_loan'],
                         ['bal_mortgage_loan_x', 'bal_mortgage_loan_y', 'bal_mortgage_loan'],
                         ['bal_current_account_x', 'bal_current_account_y', 'bal_current_account'],
                         ['bal_pension_saving_x', 'bal_pension_saving_y', 'bal_pension_saving'],
                         ['bal_savings_account_x', 'bal_savings_account_y', 'bal_savings_account'],
                        ]

        for balance_type in list_balances:
            x_dataset['{}_1'.format(balance_type[2])] = percentage_change(x_dataset[balance_type[0]],x_dataset[balance_type[1]]) 
            x_dataset['{}_2'.format(balance_type[2])] = percentage_change(x_dataset[balance_type[1]],x_dataset[balance_type[2]]) 
            x_dataset['{}_3'.format(balance_type[2])] = percentage_change(x_dataset[balance_type[0]],x_dataset[balance_type[2]]) 

            x_dataset['{}_1'.format(balance_type[2])] = x_dataset['{}_1'.format(balance_type[2])].fillna(0)
            x_dataset['{}_2'.format(balance_type[2])] = x_dataset['{}_2'.format(balance_type[2])].fillna(0)
            x_dataset['{}_3'.format(balance_type[2])] = x_dataset['{}_3'.format(balance_type[2])].fillna(0)    
            
        return x_dataset


In [7]:
# Features to preprocess

# Features to drop
drop_features = [
    'Unnamed: 0',
    'client_id', #ID not needed in the training data
    'customer_education_x', #Remove education b/c missing > 70%
    'customer_education_y', #
    'customer_education',#
]

# Duplicated Columns to drop
dup_cols = data.T.duplicated().reset_index()
dup_cols_list = dup_cols.loc[dup_cols[0], 'index'].tolist()
drop_features.extend(dup_cols_list)


# Categorical Features to Preprocess
categorical_features = [
    'customer_relationship', 'customer_relationship_y', 'customer_relationship_x', 
    'customer_occupation_code_x',
    'customer_children', 'customer_children_y', 'customer_children_x',
    'customer_gender_x',
    'area_cat',
    'customer_self_employed', 'customer_self_employed_y', 'customer_self_employed_x',
]

# Numerical Features to Preprocess
numeric_features = [
    'bal_insurance_21', 'bal_insurance_21_y', 'bal_insurance_21_x', 
    'bal_insurance_23', 'bal_insurance_23_y', 'bal_insurance_23_x',
    'cap_life_insurance_fixed_cap', 'cap_life_insurance_fixed_cap_y', 'cap_life_insurance_fixed_cap_x',
    'cap_life_insurance_decreasing_cap', 'cap_life_insurance_decreasing_cap_y', 'cap_life_insurance_decreasing_cap_x',
    'prem_fire_car_other_insurance', 'prem_fire_car_other_insurance_y', 'prem_fire_car_other_insurance_x',
    'bal_personal_loan', 'bal_personal_loan_y', 'bal_personal_loan_x',
    'bal_mortgage_loan', 'bal_mortgage_loan_y', 'bal_mortgage_loan_x',
    'bal_current_account', 'bal_current_account_y', 'bal_current_account_x',
    'bal_pension_saving', 'bal_pension_saving_y', 'bal_pension_saving_x', 
    'bal_savings_account', 'bal_savings_account_y', 'bal_savings_account_x',
    'bal_current_account_starter', 'bal_current_account_starter_y', 'bal_current_account_starter_x',
    'bal_savings_account_starter', 'bal_savings_account_starter_y', 'bal_savings_account_starter_x',
    'visits_distinct_so', 'visits_distinct_so_y', 'visits_distinct_so_x',
    'visits_distinct_so_areas', 'visits_distinct_so_areas_y', 'visits_distinct_so_areas_x',
    'customer_since_all_x', 'customer_since_bank_x', 'customer_birth_date_x',
]

In [8]:
# List of Duplicated Columns
dup_cols_list

['has_savings_account_starter_y',
 'customer_since_all_y',
 'customer_since_bank_y',
 'customer_gender_y',
 'customer_birth_date_y',
 'customer_postal_code_y',
 'customer_occupation_code_y',
 'customer_education_y',
 'customer_since_all',
 'customer_since_bank',
 'customer_gender',
 'customer_birth_date',
 'customer_postal_code',
 'customer_occupation_code',
 'customer_education']

In [9]:
# Replace NA with column mean, normalize numerical values
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('normalize', Normalizer()),
])

# Replace NA with column mode, encode categorical value to 0/1
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

In [10]:
# Pre-process pipeline that drops unnecessary features, transforms numerical and categorical values
preprocessor = ColumnTransformer(remainder='passthrough',
                                 transformers=[
                                     ('drop_columns', 'drop', drop_features),
                                     ('numeric', numeric_transformer, numeric_features),
                                     ('categorical', categorical_transformer, categorical_features)
                                ])

In [11]:
# Pipeline from preprocessing to fitting a model
pipeline = Pipeline(steps=[
    ('percent_change', PercetChange()),
    ('area_code_categorize', AreaCode()),
    ('inbetween_days', InbetweenDays()),
    ('binarize_children', ChildrenStatus()),
    ('preprocessor', preprocessor),
    ('catboost_weighted', CatBoostClassifier(verbose=False,random_state=0)),#,scale_pos_weight=2)),
])

In [12]:
# pipeline.fit(X_train, y_train)
pipeline.fit(X_over, y_over)
#pipeline.fit(X_under, y_under)

/var/folders/xc/tvrxkd8d2d71ksnxgn8wf9t40000gn/T/ipykernel_1567/3824788444.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x_dataset['{}_1'.format(balance_type[2])] = percentage_change(x_dataset[balance_type[0]],x_dataset[balance_type[1]])
/var/folders/xc/tvrxkd8d2d71ksnxgn8wf9t40000gn/T/ipykernel_1567/3824788444.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x_dataset['{}_2'.format(balance_type[2])] = percentage_change(x_dataset[balance_type[1]],x_dataset[balance_type[2]])
/var/folders/xc/tvrxkd8d2d71ksnxgn8wf9t40000

Pipeline(steps=[('percent_change', PercetChange()),
                ('area_code_categorize', AreaCode()),
                ('inbetween_days', InbetweenDays()),
                ('binarize_children', ChildrenStatus()),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_columns', 'drop',
                                                  ['Unnamed: 0', 'client_id',
                                                   'customer_education_x',
                                                   'customer_education_y',
                                                   'customer_education',
                                                   'ha...
                                                  ['customer_relationship',
                                                   'customer_relationship_y',
                                                   'customer_relationship_x',
                          

In [13]:
# With scale_pos_weight=5, minority class gets 5 times more impact and 5 times more correction than errors made on the majority class.
#catboost_5 = CatBoostClassifier(verbose=False,random_state=0,scale_pos_weight=5)

# Evaluate Model
y_pred = pipeline.predict(X_test)

print(f'Accuracy: {round(accuracy_score(y_test, y_pred),4)}')
print(f'Recall: {round(recall_score(y_test, y_pred),4)}')
print(f'ROC_AUC: {round(roc_auc_score(y_test, y_pred),4)}')
print(f'Precision: {round(precision_score(y_test, y_pred),4)}')

Accuracy: 0.9497
Recall: 0.1126
ROC_AUC: 0.5435
Precision: 0.1142


In [14]:
print(f'Accuracy: {round(accuracy_score(y_test[:250], y_pred[:250]),4)}')
print(f'Recall: {round(recall_score(y_test[:250], y_pred[:250]),4)}')
print(f'ROC_AUC: {round(roc_auc_score(y_test[:250], y_pred[:250]),4)}')
print(f'Precision: {round(precision_score(y_test[:250], y_pred[:250]),4)}')

Accuracy: 0.964
Recall: 0.2
ROC_AUC: 0.5898
Precision: 0.1667


In [15]:
pipeline.score(X_test, y_test)

0.9535321821036107

In [16]:
from sklearn.metrics import classification_report

target_names = ['Non-Churners', 'Churners']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

Non-Churners       0.97      0.97      0.97     12376
    Churners       0.11      0.11      0.11       364

    accuracy                           0.95     12740
   macro avg       0.54      0.54      0.54     12740
weighted avg       0.95      0.95      0.95     12740



In [ ]:
# Fit a LGBM Class

# from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
# Pipeline from preprocessing to fitting a model
pipeline2 = Pipeline(steps=[
    ('inbetween_days', InbetweenDays()),
    ('binarize_children', ChildrenStatus()),
    ('preprocessor', preprocessor),
    ('LGBMClassifier', LGBMClassifier(random_state=0))
#     ('logistic', LogisticRegression(random_state=0)),
])

pipeline2.fit(X_over, y_over)

# Evaluate Model
y_pred2 = pipeline2.predict(X_test)

print(f'Accuracy: {round(accuracy_score(y_test, y_pred2),4)}')
print(f'Recall: {round(recall_score(y_test, y_pred2),4)}')
print(f'ROC_AUC: {round(roc_auc_score(y_test, y_pred2),4)}')
print(f'Precision: {round(precision_score(y_test, y_pred2),4)}')